## Testing and timing of `moarchiving.BiobjectiveNondominatedSortedList`

In [1]:
import doctest
import moarchiving
# reload(moarchiving)
NA = moarchiving.BiobjectiveNondominatedSortedList
doctest.testmod(moarchiving)

/Users/hansen/gitlab/randopt/team/python/moarchiving.py:284: UserWarning: compute_hypervolume needs polishing/proof reading and more testing
  warnings.warn("compute_hypervolume needs polishing/proof reading and more testing")


TestResults(failed=0, attempted=25)

In [2]:
a = moarchiving.BiobjectiveNondominatedSortedList(
    [[-0.749, -1.188], [-0.557, 1.1076],
     [0.2454, 0.4724], [-1.146, -0.110]])
a._asserts()
a

[[-1.146, -0.11], [-0.749, -1.188]]

In [3]:
a.dominators([1, 3]) == a

True

In [4]:
a.add([-1, -3])  # return index where the element was added

1

In [5]:
a

[[-1.146, -0.11], [-1, -3]]

In [6]:
a.add([-1.5, 44])

0

In [7]:
a

[[-1.5, 44], [-1.146, -0.11], [-1, -3]]

In [8]:
a.dominates(a[0])

True

In [9]:
a.dominates([-1.2, 1])

False

In [10]:
a._asserts()

In [11]:
import numpy as np
def nondom_arch(n):
    return np.abs(np.linspace(-1, 1, 2 * n).reshape(2, n).T).tolist()
# nondom_arch(3)

## Timing of initialization

In [12]:
%timeit nondom_arch(1_000)
%timeit nondom_arch(10_000)
%timeit nondom_arch(100_000)  # just checking baseline

The slowest run took 5.84 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 131 µs per loop
1000 loops, best of 3: 878 µs per loop
100 loops, best of 3: 15.9 ms per loop


In [13]:
%timeit NA(nondom_arch(1_000))
%timeit NA(nondom_arch(10_000))
%timeit NA(nondom_arch(100_000))  # nondom_arch itself takes about 25%

1000 loops, best of 3: 737 µs per loop
100 loops, best of 3: 7.38 ms per loop
10 loops, best of 3: 82.2 ms per loop


In [14]:
randars = {}  # prepare input lists
for n in [1_000, 10_000, 100_000]:
    randars[n] = np.random.rand(n, 2).tolist()
len(NA(nondom_arch(100_000))), len(NA(randars[100_000]))

(100000, 4)

In [15]:
%timeit NA(randars[1_000])
%timeit NA(randars[10_000])
%timeit NA(randars[100_000])

1000 loops, best of 3: 731 µs per loop
100 loops, best of 3: 9.17 ms per loop
10 loops, best of 3: 185 ms per loop


In [16]:
%timeit sorted(nondom_arch(1_000))
%timeit sorted(nondom_arch(10_000))
%timeit sorted(nondom_arch(100_000))

The slowest run took 4.55 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 129 µs per loop
1000 loops, best of 3: 1.26 ms per loop
10 loops, best of 3: 20.3 ms per loop


In [17]:
%timeit sorted(randars[1_000])
%timeit sorted(randars[10_000])
%timeit sorted(randars[100_000])

1000 loops, best of 3: 330 µs per loop
10 loops, best of 3: 5.94 ms per loop
10 loops, best of 3: 124 ms per loop


In [18]:
%timeit list(randars[1_000])
%timeit list(randars[10_000])
%timeit list(randars[100_000])

The slowest run took 4.06 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.39 µs per loop
10000 loops, best of 3: 34.7 µs per loop
1000 loops, best of 3: 757 µs per loop


### Summary with 1e5 data
```
   1 ms `list` 
  22 ms `sorted` on sorted list
 130 ms `sorted` on unsorted list
  80 ms archive on sorted nondominated list
 190 ms archive on list which needs pruning (was 1300ms)
```

## Timing of `add`

In [19]:
%%timeit a = NA(nondom_arch(1_000))
for i in range(1000):
    a.add([ai - 1e-4 for ai in a[np.random.randint(len(a))]])
len(a)

100 loops, best of 3: 5.81 ms per loop


In [20]:
%%timeit a = NA(nondom_arch(10_000))
for i in range(1000):
    a.add([ai - 1e-4 for ai in a[np.random.randint(len(a))]])
len(a)

100 loops, best of 3: 6.62 ms per loop


In [21]:
%%timeit a = NA(nondom_arch(100_000))
for i in range(1000):
    a.add([ai - 1e-4 for ai in a[np.random.randint(len(a))]])
len(a)  # deletion kicks in and makes it 20 times slower if implemented with pop

100 loops, best of 3: 7.77 ms per loop


In [22]:
%%timeit a = NA(nondom_arch(100_000))
for i in range(1000):
    a.add([ai - 1e-8 for ai in a[np.random.randint(len(a))]])
len(a) # no deletion has taken place

100 loops, best of 3: 9.11 ms per loop


In [23]:
%%timeit a = NA(nondom_arch(1_000_000))
for i in range(1000):
    a.add([ai - 1e-4 for ai in a[np.random.randint(len(a))]])
len(a)  # deletion kicks in

1 loop, best of 3: 257 ms per loop


In [24]:
%%timeit a = NA(nondom_arch(1_000_000))
for i in range(1000):
    a.add([ai - 1e-8 for ai in a[np.random.randint(len(a))]])
len(a)

100 loops, best of 3: 11.5 ms per loop
